## imports

In [ ]:
import sys 
from collections import defaultdict
# genequest imports 
from genequest.io_handler.parser import FastaReader
from genequest.common.errors import *
from genequest.analysis.assembler import run_de_brujin


### inputs 

In [ ]:
reads = "../data/READS.fasta"
query = "../data/QUERY.fasta"
kmer_size = 20
match_score = 10
gap_score = 0
mismatch_score = -5

### Main script 

In [ ]:
# loading in the read and gene_query
entries = FastaReader(reads)
gene_query = FastaReader(query)

print("Loaded Sequence information:")
print("Query sequence length: {}".format(len(gene_query[0])))
print("Total number of reads: {}".format(entries.n_entries))
print("Scaffolds: {}".format(entries.scaffold_ids))
print("Total number of scaffolds: {}".format(entries.n_scaffolds))

# grouped_entries = entries.group_by_scaffold()


In [ ]:
data = run_de_brujin(entries, k=31)

In [ ]:
data

In [ ]:
class Node:
    """Class Node to represent a vertex in the de bruijn graph"""

    def __init__(self, lab):
        self.label = lab
        self.indegree = 0
        self.outdegree = 0


class Edge:
    def __init__(self, lab):
        self.label = lab


def construct_graph(reads, k):
    """Construct de bruijn graph from sets of short reads with k length word"""
    edges = dict()
    vertices = dict()

    for read in reads:
        i = 0
        while i + k < len(read):
            v1 = read[i : i + k]
            v2 = read[i + 1 : i + k + 1]
            if v1 in edges.keys():
                vertices[v1].outdegree += 1
                edges[v1] += [Edge(v2)]
            else:
                vertices[v1] = Node(v1)
                vertices[v1].outdegree += 1
                edges[v1] = [Edge(v2)]
            if v2 in edges.keys():
                vertices[v2].indegree += 1
            else:
                vertices[v2] = Node(v2)
                vertices[v2].indegree += 1
                edges[v2] = []
            i += 1

    return (vertices, edges)


def output_contigs(g):
    """Perform searching for Eulerian path in the graph to output genome assembly"""
    V = g[0]  # NODES
    E = g[1]  # EDGES

    # Pick starting node (the vertex with zero in degree)
    start = V.keys()[0]
    for k in V.keys():
        if V[k].indegree < V[start].indegree:
            start = k

    contig = start
    current = start
    while len(E[current]) > 0:
        # Pick the next node to be traversed (for now, at random)
        next = E[current][0]
        del E[current][0]
        contig += next.label[-1]
        current = next.label

    return contig


def print_graph(g):
    """Print the information in the graph to be (somewhat) presentable"""
    V = g[0]
    E = g[1]
    for k in V.keys():
        print(
            "name: ",
            V[k].label,
            ". indegree: ",
            V[k].indegree,
            ". outdegree: ",
            V[k].outdegree,
        )
        print("Edges: ")
        for e in E[k]:
            print(e.label)
        print()


In [ ]:
sample_reads = []
for entry in entries[:1000]:
    sample_reads.append(entry.seq)
len(sample_reads)


In [ ]:

data = construct_graph(sample_reads, k=15)
# len(sample_reads)


In [ ]:
# contig = output_contigs(nodes, edges)
nodes = data[0]
edges = data[1]

# selecting a starting node
start = list(nodes.keys())[0]
for k in nodes.keys():
    if nodes[k].indegree < nodes[start].indegree:
        start = k

contig = start
current = start
with open("assembly.log", "w") as f:
    while len(edges[current]) > 0:
        _next = edges[current][0]
        del edges[current][0]
        contig += _next.label[-1]
        current = _next.label
        f.write(f"{_next.label[-1]}\n")


In [ ]:
print_graph(data)

In [ ]:
contig

In [ ]:
class Node:
    """Class Node to represent a vertex in the de bruijn graph"""

    def __init__(self, label):
        self.label = label
        self.indegree = 0
        self.outdegree = 0


class Edge:
    def __init__(self, label):
        self.label = label

In [ ]:
k = 3
read = "TTCAGGCTCTGGCATGCATTAGAAATGTGGCTTGTTT"

edges = dict()
nodes = dict()

for i in range(len(read) - k + 1):
    edge1 = read[i:i+k]
    edge2 = read[i+1:i+k+1]
    if edge1 in edges.keys():
        nodes[edge1].outdegree += 1
        edges[edge1] += [Edge(edge2)]
    else:
        nodes[edge1] = Node(edge1)
        nodes[edge1].outdegree += 1
        edges[edge1] = [Edge(edge2)]

    if edge2 in edges.keys():
        nodes[edge2].indegree += 1
    else:
        nodes[edge2] = Node(edge2)
        nodes[edge2].indegree += 1
        edges[edge2] = []
    i += 1

data = (nodes, edges)

In [ ]:
nodes = data[0]
edges = data[1]

start = list(nodes.keys())[0]

starting_edge = []
for k in nodes.keys():
    if nodes[k].indegree == 0:
        start
         
    # if nodes[k].indegree < nodes[start].indegree:
    #     start = k

print(start)
print(edges[start])

In [ ]:
contig = start 
current = start

while len(edges[current]) > 0:
    next = edges[current][0]
    del edges[current][0]
    contig += next.label[-1]
    current = next.label
    print(nodes[current].outdegree)


In [ ]:
#bob
print(contig)

In [ ]:
edges["TCA"][0].label

In [ ]:
e1 = edges["GCT"][0].label
e2 = edges["GCT"][1].label
print(e1, e2)

# Testing graph building 

In [1]:
import sys
from collections import defaultdict

# genequest imports
from genequest.common.errors import *
from genequest.io_handler.parser import FastaReader
from genequest.io_handler.gene_io import load_contigs
from genequest.analysis.assembler import run_de_bruijn
from genequest.analysis.alignment import local_alignment

reads = "../data/READS.fasta"
query = "../data/QUERY.fasta"
kmer_size = 20
match_score = 10
gap_score = 0
mismatch_score = -5

# loading in the read and gene_query
entries = FastaReader(reads)
gene_query = FastaReader(query)

print("Loaded Sequence information:")
print("Query sequence length: {}".format(len(gene_query[0])))
print("Total number of reads: {}".format(entries.n_entries))
print("Scaffolds: {}".format(entries.scaffold_ids))
print("Total number of scaffolds: {}".format(entries.n_scaffolds))

print("\nstarting assembly")
contigs = run_de_bruijn(entries, k=20, save=True)


Loaded Sequence information:
Query sequence length: 648
Total number of reads: 124520
Scaffolds: ['2G5Z3', '2S43D']
Total number of scaffolds: 2

starting assembly


In [7]:
# time for alignment 
data = load_contigs()
# query = gene_query[0].seq

In [8]:
data

defaultdict(None,
            {'2S43D': defaultdict(None,
                         {'conting1': 'TTCAGGCTCTGGCATGCATTAGAAATGTGGCTTGTTTTT',
                          'conting2': 'GGGTGGTCCCCCTCCTTTACTTGTAACGTTGTCCTAAGTCGTTTTCTTTAGCCCATGGTGTTGGTGGGGTTCACAGAAACACCCAGAGTTCACCTGAGCCTTTAACCAATCCCAGCCCAGGGAGCCAGAGCCCAGGCACAGGTGCAGGACCACGGCAGGCCCAGTATTGGCTCCGACAGAAGCTACGGCATCCTATCGAGTGCACTGGGCTCGTGGTGGGAAGCAGGACAA',
                          'conting3': 'GGGTGGTCTCCTTTACTTGTAACTTGTCCTAAGTCGTTTCTTTAGCCCATGGTGTTGGTGGGGTTCACAGAAACACCCAGAGTTCACCTGAGCCTTTAACCAATCCCAGCCCAGGGAGCCAGAGCCCAGGCACAGGTGCAGGACCACGGCAGGCCCAGTATTGGCTCCGACAGAAGCTACGGCATCCTATCGAGTGCACTGGGCTCGTGGTGGGAAGCAGGACAA',
                          'conting4': 'CAACAGGGTTTTGGAAATTTGCCCATTTGCATGGCGAAGACCACCTCTCTCTCTCTCATCGACCTT',
                          'conting5': 'CCCCCCTCCTTTATTTTGTTGATTATTGAGTTTGGCATTCTGTTCTTGTGGCTCTCTTCTTTTGTTTCGTTTGAGGAATACTTCTTGGCTTTTTCTACTGGGCGTGAGTTTTCTTGGTCCTTGATTATTGGGTTT',
                          'conting6'